CHATBOT FOR TESTING PURPOSE

In [1]:
!pip install transformers sentence-transformers faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 25.4 MB/s eta 0:00:00


In [2]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer
import re


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
# Load a Sentence Transformer model for encoding knowledge base documents
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Load an LLM for conversational response generation
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
# Define a simple knowledge base relevant to insurance claims
knowledge_base = [
    "Insurance claims can be filed for various events like accidents, theft, and natural disasters.",
    "Provide details like policy number, claim amount, and incident description.",
    "Police reports are required for certain types of claims like accidents and theft.",
    "Photographic documentation can help support your claim.",
    "You can contact us for claim status or additional questions."
]

# Encode the knowledge base documents into embeddings
doc_embeddings = embedder.encode(knowledge_base, convert_to_tensor=True).cpu().numpy()

# Initialize FAISS index
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(doc_embeddings)


In [5]:
# Function to retrieve relevant documents based on user query
def retrieve_context(user_query, top_k=1):
    query_embedding = embedder.encode([user_query], convert_to_tensor=True).cpu().numpy()
    distances, indices = index.search(query_embedding, top_k)
    return [knowledge_base[i] for i in indices[0]]


In [6]:
# Function to extract relevant parts of the response based on the field name
def extract_relevant_input(user_input, field_name):
    if field_name == "name":
        match = re.search(r"my name is (\w+)", user_input, re.IGNORECASE)
        return match.group(1) if match else user_input.strip()

    elif field_name == "claim_amount":
        match = re.search(r"(\d+)", user_input)
        return match.group(1) if match else user_input.strip()

    elif field_name == "claim_type":
        options = ["Accident", "Theft", "Natural Disaster", "Medical", "Other"]
        for option in options:
            if option.lower() in user_input.lower():
                return option
        return user_input.strip()

    elif field_name == "police_report" or field_name == "injuries":
        if "yes" in user_input.lower():
            return "Yes"
        elif "no" in user_input.lower():
            return "No"
        return user_input.strip()

    elif field_name == "claim_date":
        match = re.search(r"\b(\d{4}-\d{2}-\d{2})\b", user_input)
        return match.group(1) if match else "Invalid date format, please enter as YYYY-MM-DD."

    elif field_name == "incident_location":
        match = re.search(r"\b([A-Za-z\s]+)\b", user_input)
        return match.group(1).strip() if match else user_input.strip()

    else:
        return user_input.strip()


In [7]:
# Function to store user responses without additional bot output
def store_user_response(user_input, responses, field_name):
    relevant_input = extract_relevant_input(user_input, field_name)
    responses[field_name] = relevant_input


In [8]:
# Chatbot conversation loop for filing an insurance claim
def insurance_claim_chatbot():
    questions = [
        ("Hello! I’ll help you file your insurance claim. Let's start with your full name.", "name"),
        ("Thanks, {name}. Can you provide your policy number?", "policy_number"),
        ("Got it. What is the type of claim you're filing today? Options are: Accident, Theft, Natural Disaster, Medical, or Other.", "claim_type"),
        ("Understood. Could you tell me the date of the incident? Please use YYYY-MM-DD format.", "claim_date"),
        ("Thanks. Please provide a brief description of the incident.", "incident_description"),
        ("How much are you claiming for this incident? Please specify the amount.", "claim_amount"),
        ("Did you file a police report for this incident? (Yes/No)", "police_report"),
        ("Thank you. Where did the incident occur? Please provide the location.", "incident_location"),
        ("Was anyone injured in the accident? (Yes/No)", "injuries"),
        ("Finally, could you tell me if you have any additional documentation? Options are: Police Report, Medical Report, Photographs, Invoices/Receipts, or None.", "additional_docs")
    ]

    responses = {}
    for question, field_name in questions:
        # Customize question based on previous responses
        if "{name}" in question and "name" in responses:
            question = question.format(name=responses["name"])

        # Print the bot's question
        print(question)
        user_input = input("User: ")

        # Store user response without any extra bot response printing
        store_user_response(user_input, responses, field_name)

    # Print a summary of collected responses at the end of the conversation
    print("\nThank you! Your claim form has been completed. Here are the details:")
    for field, answer in responses.items():
        print(f"{field.replace('_', ' ').capitalize()}: {answer}")


Instructions:

Before proceeding to the next steps, please start by interacting with the chatbot above. Begin by entering your full name to initialize the form-filling process.

If you'd like, you can start with a greeting, but note that the chatbot will only capture and process the specific information needed, like your name and other required fields, ignoring unnecessary text.
Simply respond to each question prompted by the chatbot, and it will guide you through filling in all the details for your insurance claim.

Once you complete the form, you’ll be able to finalize and review all entered information. Enjoy!

In [9]:
# Run the chatbot
insurance_claim_chatbot()


Hello! I’ll help you file your insurance claim. Let's start with your full name.
User: utsav
Thanks, utsav. Can you provide your policy number?
User: 123ddd
Got it. What is the type of claim you're filing today? Options are: Accident, Theft, Natural Disaster, Medical, or Other.
User: accident
Understood. Could you tell me the date of the incident? Please use YYYY-MM-DD format.
User: 2003-05-06
Thanks. Please provide a brief description of the incident.
User: jmcjjjdj
How much are you claiming for this incident? Please specify the amount.
User: 2000
Did you file a police report for this incident? (Yes/No)
User: yes
Thank you. Where did the incident occur? Please provide the location.
User: delhi
Was anyone injured in the accident? (Yes/No)
User: no
Finally, could you tell me if you have any additional documentation? Options are: Police Report, Medical Report, Photographs, Invoices/Receipts, or None.
User: none

Thank you! Your claim form has been completed. Here are the details:
Name: u

ACTUAL CHATBOT WITH FORM


In [10]:
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [11]:
import gradio as gr
import re


In [12]:
# Define the chatbot question flow and response storage
questions = [
    ("Hello! I’ll help you file your insurance claim. Let's start with your full name.", "name"),
    ("Thanks, {name}. Can you provide your policy number?", "policy_number"),
    ("Got it. What is the type of claim you're filing today? Options are: Accident, Theft, Natural Disaster, Medical, or Other.", "claim_type"),
    ("Understood. Could you tell me the date of the incident? Please use YYYY-MM-DD format.", "claim_date"),
    ("Thanks. Please provide a brief description of the incident.", "incident_description"),
    ("How much are you claiming for this incident? Please specify the amount.", "claim_amount"),
    ("Did you file a police report for this incident? (Yes/No)", "police_report"),
    ("Thank you. Where did the incident occur? Please provide the location.", "incident_location"),
    ("Was anyone injured in the accident? (Yes/No)", "injuries"),
    ("Finally, could you tell me if you have any additional documentation? Options are: Police Report, Medical Report, Photographs, Invoices/Receipts, or None.", "additional_docs")
]

responses = {}


In [13]:
# Function to update responses based on the question flow
def chatbot_interaction(user_input, chat_history):
    # Store user response based on the current question
    if chat_history:
        prev_question, field_name = questions[len(chat_history)//2 - 1]
        responses[field_name] = extract_relevant_input(user_input, field_name)

    # Get the next question if there are any left
    if len(chat_history)//2 < len(questions):
        question, field_name = questions[len(chat_history)//2]
        bot_response = question.format(name=responses.get("name", ""))
        chat_history.append(("user", user_input))  # Add user input to history
    else:
        bot_response = "Thank you! Your claim has been successfully submitted."

    # Update chat history with bot response
    chat_history.append(("bot", bot_response))
    return chat_history, responses


In [14]:
# Function to extract the relevant part of the user input based on the question
def extract_relevant_input(user_input, field_name):
    # Strip leading/trailing whitespaces and match only the relevant part of the input
    user_input = user_input.strip().lower()

    if field_name == "name":
        match = re.search(r"my name is (\w+)", user_input, re.IGNORECASE)
        return match.group(1) if match else user_input.split()[0]  # Take only the first word as name

    elif field_name == "claim_amount":
        match = re.search(r"(\d+)", user_input)
        return match.group(1) if match else user_input.strip()  # Only the numeric value

    elif field_name == "claim_type":
        options = ["accident", "theft", "natural disaster", "medical", "other"]
        for option in options:
            if option in user_input:
                return option.capitalize()  # Return the matched claim type
        return user_input.strip()

    elif field_name == "police_report" or field_name == "injuries":
        if "yes" in user_input:
            return "Yes"
        elif "no" in user_input:
            return "No"
        return user_input.strip()

    elif field_name == "claim_date":
        match = re.search(r"\b(\d{4}-\d{2}-\d{2})\b", user_input)
        return match.group(1) if match else "Invalid date format, please enter as YYYY-MM-DD."

    elif field_name == "incident_location":
        match = re.search(r"\b([A-Za-z\s]+)\b", user_input)
        return match.group(1).strip() if match else user_input.strip()

    elif field_name == "additional_docs":
        # Extract key words for additional documents
        docs = ["police report", "medical report", "photographs", "invoices/receipts", "none"]
        for doc in docs:
            if doc in user_input:
                return doc.capitalize()
        return user_input.strip()

    return user_input.strip()


In [15]:
# Function to display final collected data in the form
def collect_claim_data():
    return [responses.get(field, "") for _, field in questions]


In [16]:
# Function for the success message after submission
def finalize_submission():
    return "Thank you for visiting! Your claim has been successfully submitted."


In [17]:
# Gradio UI layout
with gr.Blocks() as demo:
    gr.Markdown("### Insurance Claim Form & Chatbot")

    with gr.Row():
        with gr.Column():
            # Chatbot interface
            chatbot = gr.Chatbot()
            user_message = gr.Textbox(placeholder="Type your response here...")

            # Finalize claim details button under chatbot
            finalize_button = gr.Button("Finalize Claim Details")

        with gr.Column():
            # Claim form fields (editable manually by the user)
            name = gr.Textbox(label="Full Name", value="", interactive=True)
            policy_number = gr.Textbox(label="Policy Number", value="", interactive=True)
            claim_type = gr.Textbox(label="Claim Type", value="", interactive=True)
            claim_date = gr.Textbox(label="Claim Date (YYYY-MM-DD)", value="", interactive=True)
            incident_description = gr.Textbox(label="Incident Description", value="", interactive=True)
            claim_amount = gr.Textbox(label="Claim Amount", value="", interactive=True)
            police_report = gr.Textbox(label="Police Report Filed?", value="", interactive=True)
            incident_location = gr.Textbox(label="Incident Location", value="", interactive=True)
            injuries = gr.Textbox(label="Injuries?", value="", interactive=True)
            additional_docs = gr.Textbox(label="Additional Documentation", value="", interactive=True)

            # Submit details button under form
            submit_button = gr.Button("Submit Details")

    # Set up interaction for the chatbot
    user_message.submit(
        fn=chatbot_interaction,
        inputs=[user_message, chatbot],
        outputs=[chatbot, gr.State()]
    )

    # Collect form data in one click after Finalize button
    finalize_button.click(
        fn=collect_claim_data,
        inputs=[],
        outputs=[name, policy_number, claim_type, claim_date, incident_description, claim_amount, police_report, incident_location, injuries, additional_docs]
    )

    # After clicking Submit Details, show the next page with success message
    submit_button.click(
        fn=finalize_submission,
        inputs=[],
        outputs=gr.HTML()
    )

demo.launch()
#click on the text box and press enter without writting anything on it this will start the chatbox


/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://07894b92849899ec44.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Steps to Use the Insurance Claim Chatbot and Form

1)Press the Link
Click the link above to open the application interface.

2)Form and Chatbot Layout
On the interface, you'll see a form on the right and a chatbot on the left.

3)Activate the Chatbot
Click the text box under the chatbot and simply press Enter without entering any response. This will start the chatbot interaction.

4)Answer All Chatbot Questions
The chatbot will guide you through the questions one by one. Enter your responses as prompted.

5)Finalize Claim Details
Once you've answered all questions, click the "Finalize Claim Details" button located under the chatbot. This will automatically fill in the form on the right with your responses.

6)Submit the Form
Review the form details. If everything looks good, click the "Submit" button at the bottom of the form. Your details will be submitted successfully.















